### starting a basic sequence to sequence LSTM model 

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings('ignore')


2024-05-31 11:07:37.238644: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-31 11:07:37.267068: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-31 11:07:37.680417: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# creating a sintetic dataframe

In [2]:
num_days = 365


start_date = datetime(2023, 1, 1)


dates = [start_date + timedelta(days=i) for i in range(num_days)]


sales = np.random.randint(10, 100, size=num_days)
sales1 = np.random.randint(9, 20, size=num_days)

df = pd.DataFrame({'data': dates, 'vendas': sales,'valor': sales})

df = df.reset_index().rename(columns={"index":"time_idx"})

df = df[['time_idx','vendas','valor']]
df.head()

,time_idx,vendas,valor
0,0,38,38
1,1,59,59
2,2,68,68
3,3,90,90
4,4,81,81


In [3]:
for col in df.select_dtypes(include=['int']):
    df[col] = df[col].astype(float)

In [4]:
datas = df.time_idx
X = df.iloc[:,1:]

In [5]:
X

,vendas,valor
0,38.0,38.0
1,59.0,59.0
2,68.0,68.0
3,90.0,90.0
4,81.0,81.0
...,...,...
360,52.0,52.0
361,38.0,38.0
362,32.0,32.0
363,92.0,92.0


In [6]:
X.shape

(365, 2)

In [7]:
datas = np.expand_dims(datas, axis=0)
datas = np.expand_dims(datas, axis=-1)
datas.shape

(1, 365, 1)

In [8]:
X = np.expand_dims(X, axis=0)
X.shape

(1, 365, 2)

In [9]:
lstm_encoder = tf.keras.layers.LSTM(units=64,
                            return_state = True,
                            name = 'LSTM_encoder')

lstm_decoder = tf.keras.layers.LSTM(units=64,
                             return_sequences=True, 
                             return_state=True,
                             name = 'LSTM_decoder')

2024-05-31 11:07:48.202800: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-05-31 11:07:48.202821: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] retrieving CUDA diagnostic information for host: Hogwarts
2024-05-31 11:07:48.202824: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:141] hostname: Hogwarts
2024-05-31 11:07:48.202964: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:165] libcuda reported version is: 555.42.2
2024-05-31 11:07:48.202976: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:169] kernel reported version is: 555.42.2
2024-05-31 11:07:48.202978: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:248] kernel version seems to match DSO: 555.42.2


In [10]:
a,b,c = lstm_encoder(datas)

x,_,_ = lstm_decoder(X,initial_state=[b,c])

In [17]:
class Encoder_layer(tf.keras.layers.Layer):
    def __init__(self, num_untis:int):
        super(Encoder_layer,self).__init__()


        self.LSTM_encoder = tf.keras.layers.LSTM(units=num_untis,
                                                 return_state = True,
                                                 name = 'LSTM_encoder')
        
    def call(self,inputs):
        
        output_encoder, state_h, state_c = self.LSTM_encoder(inputs)

        return output_encoder, state_h, state_c
    
teste = Encoder_layer(64)

x,a,b = teste(datas)

In [26]:
class Decoder_layer(tf.keras.layers.Layer):
    def __init__(self, num_untis:int):
        super(Decoder_layer, self).__init__()

        self.LSTM_decoder = tf.keras.layers.LSTM(units=num_untis,
                                                 return_sequences=True,
                                                 return_state = True,
                                                 name = 'LSTM_decoder')

        self.Dense_output = tf.keras.layers.Dense(2)
    def call(self,inputs,encoder_states):


        output_decoder, state_h, state_c = self.LSTM_decoder(inputs,initial_state=encoder_states)
        output = self.Dense_output(output_decoder)

        return output

In [27]:
teste1 = Decoder_layer(64)

x = teste1(X, encoder_states =[a,b])

In [28]:
class Seq_to_Seq_LSTM(tf.keras.Model):
    def __init__(self, num_units:int):

        super().__init__()
        self.num_units = num_units


        self.encoder_layer = Encoder_layer(self.num_units)
        self.decoder_layer = Decoder_layer(self.num_units)


    def call(self,inputs):

        enc,dec= inputs
       
        _, state_h, state_c = self.encoder_layer(enc)

        output_decoder = self.decoder_layer(dec, encoder_states = [state_h, state_c])

        return output_decoder
    




In [29]:

model = Seq_to_Seq_LSTM(num_units=64)

model.compile(optimizer='rmsprop', loss='mae')


model.fit([datas,X],
          X,epochs=100,verbose=False)